In [1]:
# from selenium import webdriver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # webdrv-manager 패키지 다운로드
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import urllib.request
import requests
from tqdm import tqdm
import os
from datetime import datetime


In [2]:

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.pinterest.co.kr')
driver.implicitly_wait(1)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - There is no [mac64] chromedriver for browser 85.0.4183 in cache
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/choiswonspec/.wdm/drivers/chromedriver/mac64/85.0.4183.87]


In [3]:
# 로그인 하기

driver.find_element_by_xpath('//*[@id=\"__PWS_ROOT__\"]/div[1]/div/div/div/div[1]/div[1]/div[2]/div[2]/button/div').click()
driver.implicitly_wait(1)
time.sleep(1)


In [ ]:
my_id = 'csw31915@daum.net'

my_password = 'welcome2019!'

driver.find_element_by_xpath('//*[@id=\"email\"]').send_keys(my_id)
driver.find_element_by_xpath('//*[@id=\"password\"]').send_keys(my_password)
driver.find_element_by_xpath('//*[@id=\"__PWS_ROOT__\"]/div[1]/div/div/div/div[1]/div[2]/div[2]/div/div/div/div[1]/div/div/div/div[3]/form/div[5]/button/div').click()
time.sleep(2)

In [ ]:
# keyword, color_name, pk 가져오기
df_61 = pd.read_excel("./new_keycolor_query_200806.xlsx", sheet_name='61')
df_62 = pd.read_excel("./new_keycolor_query_200806.xlsx", sheet_name='62')
df_64 = pd.read_excel("./new_keycolor_query_200806.xlsx", sheet_name='64')
df_61['from'] = 61
df_62['from'] = 62
df_64['from'] = 64

* 검색어 ( search_query ) -> google_query
* kl_name (키토크) -> color_name
* 이미지 저장 path -> direct + 파일명
* 이미지 url -> 
* category_uid -> sheet 이름

# 파일명 양식

{date}/{resource}/{kl_pk}_{kl_name}_{category_uid}_{cnt}.jpg
* date : yymmdd
* cnt: 0부터 download 받은 순서(크게 관계없음)

In [ ]:
################################## 코드 끊겨서 다시 돌릴 때
'''df_62 = df_62.loc[200:].reset_index().drop(columns = 'index', axis = 1)
df_62
'''

In [ ]:
search_query = []
kl_name = []
path = []
url_column = []
category_uid = []
kl_pk = [
    
]
# 파일명에 넣을 출처 이름
resource = 'pinterest'

#### 폴더명에 넣을 현재 날짜 
#date = datetime.today().strftime("%y%m%d")
date = '200825'

for df in tqdm([df_62]):#, df_62, df_64]):  # 각 엑셀 시트를 가져와서 반복문 
    
    #### 검색을 할 때는 언더바 _ 가 있어야 한다.
    df['google_query'] = df['google_query'].str.replace(" ", "_")
    df['color_name'] = df['color_name'].str.replace(" ","_")

    # 폴더 명에 넣을 sheet이름은 df['from'] 에서 가져오면 된다.
    where = df['from'][0]
    
    for idx in tqdm(range(len(df))):
        # 파일명에 넣을 google_query.. 실제 검색할 내용
        word = df['google_query'][idx]
        
        # 파일명에 넣을 color_name
        color = df['color_name'][idx]
        
        # 파일명에 넣을 pk
        pk = df['pk'][idx]
        
        # 사이트 접속
        driver.get('https://www.pinterest.co.kr')
        time.sleep(1.5)
        
        # 검색어에 입력후 검색
        a = driver.find_element_by_xpath('//*[@id="searchBoxContainer"]/div/div/div[2]/input')
        a.send_keys(word)
        a.send_keys(Keys.ENTER)
        time.sleep(1.5)
        
        
         # 스크롤 내리기... pinterest 사이트 특성상 스크롤을 내릴 때마다 html 수집하여 이미지 url을 수집해야한다.. 
        b = driver.find_element_by_tag_name('body')
        
        image_list = []
        escape_num = 0
        while len(image_list)<501 and escape_num < 50:
            #for i in range(3):
            b.send_keys(Keys.END)
            time.sleep(1.5)

            # 해당 페이지 html 코드에서 이미지 url가져오기
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            k = soup.find_all("img")
            
            time.sleep(1)

            for j in range(len(k)):
                image_list.append(k[j]['src'])
            


            image_list = list(set(image_list))
            time.sleep(0.1)
            escape_num += 1
            time.sleep(1)
            
        
        # 고화질로 얻기 위해 url 의 일부를 변경
        temp = pd.DataFrame(image_list)
        temp[0] = temp[0].str.replace("236x","564x")
        url_list = temp[0].to_list()
        
        # 폴더 만들기
        os.makedirs(f'/Users/choiswonspec/pinterest/{date}/{resource}', exist_ok = True)
        #u = os.path.join('/Users/choiswonspec/pinterest',word)
        #os.mkdir(u)
        {date}/{resource}/{kl_pk}_{kl_name}_{category_uid}_{cnt}.jpg
        
        count = 0
        for url in url_list:
            try:
                res = requests.get(url, verify=False, stream=True)
                rawdata = res.raw.read()
                direct = f'/Users/choiswonspec/pinterest/{date}/{resource}'
                with open(os.path.join(direct, str(pk) +"_"+ color +"_"+ where +"_"+ str(count) + '.jpg'), 'wb') as f:
                    f.write(rawdata)
                    
                    time.sleep(0.1)
                ###### db data 저장
                path_full = f'{date}/{resource}/' + str(pk) +"_"+ color +"_"+ where +"_"+ str(count) + '.jpg'
                count += 1
                search_query.append(word)
                kl_name.append(color)
                path.append(path_full)
                url_column.append(url)
                category_uid.append(where)
                kl_pk.append(pk)
                time.sleep(0.1)
                
                
                
                
            except Exception as e:
                print('Failed to write rawdata.')
                print(e)
            if count == 501:
                break
        time.sleep(1)

    
    

In [ ]:
result = pd.DataFrame(list(zip(search_query, kl_name, path, url_column, category_uid)),
                          columns=['search_query', 'kl_name', 'path', 'url_column','category_uid'])
result

In [ ]:
result.to_excel('./pinterest_to_db_62_last.xlsx', index=False, encoding='utf-8')